In [111]:
import numpy as np


In [112]:
# Initial Parameters

no_of_timesteps = 1000
no_of_lengthsteps = 10

total_time = 60
L = 2.286/1000
initial_temperature_c = 150
enclosure_temperature_c = 30
initial_temperature = initial_temperature_c+273
enclosure_temperature = enclosure_temperature_c+273

timestep = total_time/no_of_timesteps
lengthstep = L/no_of_lengthsteps


HT_density = 953
HT_conductivity = 0.105
HT_heatcapacity = 2.05*1000
HT_alpha = HT_conductivity / HT_density / HT_heatcapacity
HT_Fo = HT_alpha*timestep/(lengthstep**2)
SB  = 5.6704*(10**-8)



In [113]:
#Discretization


length_nodes = np.linspace(0, L, no_of_lengthsteps)
time_nodes = np.linspace(0,total_time, no_of_timesteps)
temperature_array = np.zeros((no_of_timesteps+1, no_of_lengthsteps))
temperature_array[0:2,:] = initial_temperature
# print(temperature_array)
# print(length_nodes)



In [114]:
#Do the computation
for i,t in enumerate(time_nodes[1:], start=1):
    temperature_array[i,0] = ((2*timestep*HT_conductivity/(lengthstep**2)/HT_density/HT_heatcapacity)*(temperature_array[i-1,1] - temperature_array[i-1,0])) + temperature_array[i-1,0]

    for j,x in enumerate(length_nodes[1:-1], start=1):
        temperature_array[i,j]= HT_Fo*(temperature_array[i-1,j+1] + temperature_array[i-1,j-1]) + (1-2*HT_Fo)*(temperature_array[i-1,j])

    temperature_array[i,-1] = (2*timestep*SB/HT_density/HT_heatcapacity/lengthstep)*((enclosure_temperature)**4 - (temperature_array[i-1,-1])**4) + (2*HT_Fo*(temperature_array[i-1,-2] - temperature_array[i-1,-1])) + temperature_array[i-1,-1]

print(temperature_array[:-1]-273)
print(HT_Fo)
    

[[150.         150.         150.         ... 150.         150.
  150.        ]
 [150.         150.         150.         ... 150.         150.
  149.6406323 ]
 [150.         150.         150.         ... 150.         149.97782415
  149.32727185]
 ...
 [137.01952713 136.89608339 136.52614192 ... 131.04711247 129.24990452
  127.23059869]
 [137.00429221 136.88087252 136.5110032  ... 131.03305155 129.23619931
  127.21729346]
 [136.98906025 136.86566461 136.49586744 ... 131.01899344 129.22249683
  127.20399085]]
0.061707980788344344
